In [1]:
from google.colab import files
  
  
uploaded = files.upload()

Saving mydata.csv to mydata.csv


In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import time

#%%

import nltk
nltk.download('stopwords')

#%%

# printing the stopwords in English
print(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [3]:
data=pd.read_csv('mydata.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31428 entries, 0 to 31427
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       31428 non-null  int64 
 1   title    31428 non-null  object
 2   text     31428 non-null  object
 3   subject  31428 non-null  object
 4   date     31428 non-null  object
 5   label    31428 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


In [4]:
data1=data.drop('Id', axis=1)
data1=data.drop('date', axis=1)
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31428 entries, 0 to 31427
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       31428 non-null  int64 
 1   title    31428 non-null  object
 2   text     31428 non-null  object
 3   subject  31428 non-null  object
 4   label    31428 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.2+ MB


In [5]:
# merging the author name and news title
data1['content'] = data1['title']+' '+data1['text']
data1.head()


,Id,title,text,subject,label,content
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, the chair of Donald Trump s cam...",News,1,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How very charitable of the Lutheran and Cath...,left-news,1,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...
2,26810,Trump’s Excuse For All His Sexist Statements ...,In case you ve been living under a rock for yo...,News,1,Trump’s Excuse For All His Sexist Statements ...
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court on Friday sentenced ...,worldnews,0,'Jihadi Gran' gets 10 years after joining son ...
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government will s...,worldnews,0,New Czech government to seek confidence vote o...


In [6]:
# separating the data & label
X = data1.drop(columns='label', axis=1)
Y = data1['label']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31428 entries, 0 to 31427
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       31428 non-null  int64 
 1   title    31428 non-null  object
 2   text     31428 non-null  object
 3   subject  31428 non-null  object
 4   content  31428 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


In [7]:

port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [8]:
data1['content'] = data1['content'].apply(stemming)


In [9]:
data1.info()
data1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31428 entries, 0 to 31427
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       31428 non-null  int64 
 1   title    31428 non-null  object
 2   text     31428 non-null  object
 3   subject  31428 non-null  object
 4   label    31428 non-null  int64 
 5   content  31428 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


,Id,title,text,subject,label,content
0,13970,WATCH: Paul Manafort Gets DESTROYED On CNN Fo...,"Paul Manafort, the chair of Donald Trump s cam...",News,1,watch paul manafort get destroy cnn defend tru...
1,41668,REFUGEE BUSINESS IS CASH COW FOR LUTHERAN CHAR...,How very charitable of the Lutheran and Cath...,left-news,1,refuge busi cash cow lutheran chariti mi state...
2,26810,Trump’s Excuse For All His Sexist Statements ...,In case you ve been living under a rock for yo...,News,1,trump excus sexist statement prove realli want...
3,30967,'Jihadi Gran' gets 10 years after joining son ...,PARIS (Reuters) - A court on Friday sentenced ...,worldnews,0,jihadi gran get year join son syria pari reute...
4,26072,New Czech government to seek confidence vote o...,PRAGUE (Reuters) - The Czech government will s...,worldnews,0,new czech govern seek confid vote januari pm p...


In [10]:
#separating the data and label
X = data1['content'].values
Y = data1['label'].values

In [11]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm=SVC()

In [14]:
param_grid = {'C': [0.1, 1],
              'gamma': [1, 0.1],
              'kernel': ['rbf']}
  
grid = GridSearchCV(svm, param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.970 total time= 8.2min
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.972 total time= 8.3min
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.971 total time= 8.2min
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.971 total time= 8.2min
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.977 total time= 8.3min
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.956 total time=10.0min
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.959 total time=10.0min
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.961 total time=10.0min
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.958 total time=10.0min
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.966 total time=10.0min
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.991 total time=12.4min
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;,

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1], 'gamma': [1, 0.1], 'kernel': ['rbf']},
             verbose=3)

In [15]:
# Single best score achieved across all params (k)
print(grid.best_score_)

# Dictionary containing the parameters (k) used to generate that score
print(grid.best_params_)

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print(grid.best_estimator_)

0.9903348723782333
{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
SVC(C=1, gamma=1)


In [16]:
classifier =SVC(C=1,kernel='rbf',gamma=1)


In [17]:
classifier.fit(X_train,y_train)

SVC(C=1, gamma=1)

In [18]:
pred=classifier.predict(X_test)


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3044
           1       1.00      0.99      0.99      3242

    accuracy                           0.99      6286
   macro avg       0.99      0.99      0.99      6286
weighted avg       0.99      0.99      0.99      6286

